<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#ЧАСТЬ-2" data-toc-modified-id="ЧАСТЬ-2-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>ЧАСТЬ 2</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
! pip install -q pandarallel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.8 MB/s eta 0:00:00


In [ ]:
! pip install torch

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00


In [ ]:
! pip install -U spacy

In [ ]:
! python -m spacy download en_core_web_sm

2023-09-24 16:24:21.046610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 16:24:22.898415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-24 16:24:25.765911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-24 16:24:25.766646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
from transformers import BertTokenizer, BertModel
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Подготовка

In [4]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [7]:
    try:
        df = pd.read_csv('/toxic_comments.csv')
    except:
        df = pd.read_csv('datasets/toxic_comments.csv')

In [8]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Что такое Unnamed?

In [9]:
df['check'] = df['Unnamed: 0'] - df.index

In [10]:
df[df['check'] != 0]

,Unnamed: 0,text,toxic,check
6080,6084,"""::I'll alos be looking in to see how this is ...",0,4
6081,6085,"""\n\nThe Ezekiel passage is quoted in the Molo...",0,4
6082,6086,Thank you for experimenting with Wikipedia. Y...,0,4
6083,6087,Any complaints with that as the new wording?,0,4
6084,6088,"""\nI also disagree with the merge as Strength ...",0,4
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,159
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,159
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,159
159290,159449,And it looks like it was actually you who put ...,0,159


Что-то похожее на второй индекс, но явно не несущее полезной нам информации. удаляем

In [11]:
df = df.drop(['Unnamed: 0','check'], axis=1)

In [12]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [14]:
df['toxic'].sum()

16186

С положительным классом (toxic) у нас всего 10% текстов в выборке

Для создания модели и ее тестировании возьмем только часть выборки для уменьшения времени расчетов

In [15]:
df_cut= df.sample(15000)

In [16]:
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 53435 to 145570
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15000 non-null  object
 1   toxic   15000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 351.6+ KB


In [17]:
df_cut['toxic'].sum()

1487

создадим Bert токенизатор и модель Bert для создания embeddings

In [18]:
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
model = BertModel.from_pretrained("unitary/toxic-bert")

In [19]:
from torch.utils.data import Dataset, DataLoader

Напишем кастомный класс датасета который будет "выдавать" токены на вход модели для эмбеддинга

In [20]:
class ToxicDataset(Dataset):



     def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int):
         self.data = data
         self.tokenizer = tokenizer
         self.max_token_len = max_token_len

     def __len__(self):
         return len(self.data)

     def __getitem__(self, index: int):
         data_row = self.data.iloc[index]
         sent = data_row['text']

         encoding = self.tokenizer.encode_plus(

                                                sent,
                                                add_special_tokens=True,
                                                max_length=self.max_token_len,
                                                return_token_type_ids=False,
                                                padding="max_length",
                                                truncation=True,
                                                return_attention_mask=True,
                                                return_tensors='pt',

                                                )

         return dict(
             input_ids=encoding["input_ids"].flatten(),
             attention_mask=encoding["attention_mask"].flatten(),)


In [21]:
data_dataset = ToxicDataset(df_cut,
                            tokenizer=tokenizer,
                            max_token_len=512)

проверим:

In [22]:
data_dataset[0]

{'input_ids': tensor([  101,  1042,  9148,  2860,  1010,  2004,  1996,  3559,  2040,  2988,
          2023,  5310,  1010,  1045,  2903,  2023,  2089,  5262,  2031,  2042,
          1037, 24216,  1997,  3343,  2011,  1037,  2047,  1010,  2512,  3128,
          2394,  4092,  5310,  1010,  1998,  1045,  2052,  2022,  7929,  2007,
          2460,  7406,  2010,  3796,  2000,  1005,  2051,  2366,  1005,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

Датасет работает мы получаем маску и токены

Создадим dataloader для выдачи батчей с токенами:

In [23]:
data_dataloader = DataLoader(data_dataset, batch_size=100, num_workers=0, shuffle=False)

In [24]:
from tqdm import tqdm

Получим embeddings:

In [52]:
embeddings = []
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") # переводим на GPU
model.to(device).eval()
with tqdm(total=len(data_dataloader)) as pbar:
     with torch.no_grad(): # отключаем вычисление градиентов
        for batch in data_dataloader:

             input_ids = batch['input_ids'].to(device)
             attention_mask = batch['attention_mask'].to(device)

             batch_embeddings = model.forward(input_ids, attention_mask) #передаем токены и маску в bert и получаем эмбединги

            #используем CLS pooling -  берем эмбеддинг только первого токена CLS чтобы захватить общий контекст всей последовательности
             embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
             pbar.update(1)

100%|██████████| 150/150 [09:10<00:00,  3.67s/it]


In [53]:
features = np.concatenate(embeddings)

## Обучение

In [54]:
features.shape

(15000, 768)

In [55]:
train_x, test_x, train_y, test_y= train_test_split(features, df_cut['toxic'], test_size=0.1, stratify=df_cut['toxic'], random_state=2022)

In [56]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((13500, 768), (13500,), (1500, 768), (1500,))

Все готово к обучению и тестрованию

In [57]:
model_lg = LogisticRegression(solver='liblinear')

In [58]:
cross_val_score(model_lg,train_x,train_y,scoring='f1',n_jobs=-1)

array([0.93084112, 0.93040293, 0.94074074, 0.94423792, 0.94517958])

На валидации получился неплохой результат!

In [59]:
model_lg.fit(train_x, train_y)

LogisticRegression(solver='liblinear')

In [60]:
predict = model_lg.predict(test_x)

In [61]:
print(f1_score(test_y, predict))

0.9473684210526316


In [62]:
print(precision_score(test_y, predict))

0.9290322580645162


In [63]:
print(recall_score(test_y, predict))

0.9664429530201343


### ЧАСТЬ 2

Загрузим модель spacy "en_core_web_sm"

In [64]:
nlp=spacy.load("en_core_web_sm")

Запишем знаки пунктуации

In [65]:
punctuations = string.punctuation

Запишем стоп слова

In [191]:
stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)

Напишем токенизатор который будет лематтизировать слова  и удаляя стоп слова и знаки пунктуации

In [179]:
def spacy_tokenizer(sentence):

    # создаем токены
    mytokens = nlp(sentence)

    # лематизируем токены

    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # удаляем пунктуацию
    mytokens = [ word for word in mytokens if word not in punctuations]

    # возвращаем токены
    return ' '.join(mytokens)

Пробуем

In [180]:
df['text'].iloc[7]

"Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned."

In [181]:
spacy_tokenizer(df['text'].iloc[7])

'your vandalism to the matt shirvington article have be revert please do not do it again or you will be ban'

Работает

In [153]:
from sklearn.base import TransformerMixin

Создадим трансформер для лемматизации текстов на базе функции spacy_tokenizer

In [147]:

class Lemmatizer(TransformerMixin):


  def fit(self, X, y=None): # y is required even though we don't use it
    return self


  def transform(self, X):
    return X.apply(spacy_tokenizer)

Проверим:

In [154]:
subset = df['text'].iloc[0:10]

In [155]:
lemmatize = Lemmatizer()

In [156]:
lemms = lemmatize.fit_transform(subset)

In [157]:
lemms

0    explanation edit username hardcore metallica f...
1    d'aww match background colour seemingly stick ...
2    hey man try edit war guy constantly remove rel...
3    real suggestion improvement wonder section sta...
4                        sir hero chance remember page
5                       congratulation use tool · talk
6                                  cocksucker pis work
7        vandalism matt shirvington article revert ban
8    sorry word nonsense offensive intend write art...
9                 alignment subject contrary dulithgow
Name: text, dtype: object

In [152]:
df['text'].iloc[0:10]

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: text, dtype: object

Работает

Инициируем TfidfVectorizer

In [194]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

Разобьем тексты на train и text. Сделаем стратификацию для соблюдения баланса классов в train и test

In [219]:
df_sample = df.sample(60000)

In [220]:
X = df_sample['text'] # The features we want to analyse
ylabels = df_sample['toxic']  # The labels, in this case feedback

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size = 0.2, random_state = 2022, stratify=ylabels)
print(f'X_train dimension: {X_train.shape}')
print(f'y_train dimension: {y_train.shape}')
print(f'X_test dimension: {X_test.shape}')
print(f'y_test dimension: {y_test.shape}')

X_train dimension: (48000,)
y_train dimension: (48000,)
X_test dimension: (12000,)
y_test dimension: (12000,)


In [221]:
y_test.sum()

1217

In [222]:
y_train.sum()

4867

Создадим два pipline'a первый будет очищать тексты и переводть слова векторную форму. Второй будет считать tf_idf и в нем же будет классификатор. Я решил разделить это чтобы подбирать GridSearch параметры только во втором пайплайне и не проходить процесс векторизации каждый новый раз с новым параметром делая грид серч по всему пайплайну

In [223]:
classifier = LogisticRegression()

In [224]:
pipe = Pipeline([("lemmas", lemmatize),
                  ("tfidf", tf_idf)
                 ]
                )

In [225]:
X_train = pipe.fit_transform(X_train)

Трансформируем первым пайплайном train выборку запишем результат в переменную pipe1_output

Создадим сетку параметров Лог регрессии для GridSearch

In [215]:
params = {
            'solver' : ['liblinear','saga'],
            'penalty' : ['l2', 'l1','elasticnet'],
            'C' :  [10, 1.0, 0.1, 0.01,0.001]
            }

Инициируем и запустим GridSearch

In [226]:
lg_gscv = GridSearchCV(classifier, params, scoring='f1',n_jobs=-1,verbose=5, refit=True)

In [227]:
lg_gscv.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
50 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 64, in _check_solver
    raise ValueError(
ValueError: Only

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [10, 1.0, 0.1, 0.01, 0.001],
                         'penalty': ['l2', 'l1', 'elasticnet'],
                         'solver': ['liblinear', 'saga']},
             scoring='f1', verbose=5)

Выведим лучшую метрику и лучшие параметры

In [228]:
lg_gscv.best_score_

0.754020967907505

In [229]:
lg_gscv.best_params_

{'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}

In [231]:
lg_model = lg_gscv.best_estimator_

Проверим работу на тестовой выборке

In [232]:
X_test = pipe.transform(X_test)

In [233]:
predicted = lg_model.predict(X_test)

In [234]:
print('Logistic Regression f1:', f1_score(y_test, predicted))
print('Logistic Regression Precision:', precision_score(y_test, predicted))
print('Logistic Regression Recall:', recall_score(y_test, predicted))

Logistic Regression f1: 0.7439134355275023
Logistic Regression Precision: 0.8241758241758241
Logistic Regression Recall: 0.6778964667214462


## Выводы

Получилось попробывать базовые подходы в NLP.

Задача решалась двумя путями:

1) получение признаков с помощью предобученной модели toxic-bert (нейронка) и затем простая линейная классификация.

2) В качестве признаков использовался tf-idf. В предварительная предобработке использовалась лемматизация spacy


Результат метрики f1 у первого подхода отличный - f1 ~0.95, получился очень точный классификатор несмотря на то что использовалось всего лишь 10% текстов.

Метрики классификации  второго подхода (tf-idf) почти достигают целевых f1>=0.75 (с целью ускорения расчетов были использованы менее половины предоставленных текстов). Метрика полноты значительно хуже чем точность: модель делает много FN ошибок, то есть модель "пропускает" много токсичных текстов